In [ ]:
# Author       : Shefali Mangal
# E-Mail       : shefalimangal24@gmail.com
# Contact      : +91-9165001002
# Designation  : Software Developer
# Decision Tree for Financial Loam EMI default detection
# Data         : bank.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import datasets
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
%matplotlib inline

In [ ]:
# load data file
bank = pd.read_csv("/kaggle/input/bankbalanced/bank.csv")
bank.head()

In [ ]:
#check if data set contains null values - Nothing found!

bank[bank.isnull().any(axis=1)].count()

In [ ]:
bank.describe()

In [ ]:
#box-plot for age
#a = sns.boxplot(x=bank["age"])
#a = sns.boxplot(bank["age"])
a = sns.boxplot(bank.age)

In [ ]:
#Distribution of age


#sns.distplot(bank.age, bins=100)
#sns.distplot(bank["age"], bins=100)
sns.distplot(bank.age, bins=200)

In [ ]:
#box-plot for duration
a = sns.boxplot(bank["duration"])

In [ ]:
#Distribution of duration
sns.distplot(bank["duration"], bins=200)

# Convert categorical data

In [ ]:
bank_data = bank.copy()
print(bank_data)

# EDA

In [ ]:
# Explore People who made a deposit Vs Job category

In [ ]:
for col in bank.columns:
    print(col)

In [ ]:
jobs = ['management','blue-collar','technician','admin.','services','retired','self-employed','student',
        'unemployed','entrepreneur','housemaid','unknown']

for j in jobs:
    print("{:15} : {:5}".format(j, len(bank_data[(bank_data.deposit == "yes") & (bank_data.job == j)])))

In [ ]:
# Different types of job categories and their counts

In [ ]:
bank_data.job.value_counts()

In [ ]:
# Combine similar jobs into categiroes

In [ ]:
bank_data['job'] = bank_data['job'].replace(['management','amin.'], 'white-collar')
bank_data['job'] = bank_data['job'].replace(['services','housemaid'], 'pink-collar')
bank_data['job'] = bank_data['job'].replace(['retired','student', 'unemployed', 'unknown'], 'other')

In [ ]:
bank_data.job.value_counts()

-------poutcome--------------

In [ ]:
print(bank_data.poutcome)

In [ ]:
bank_data.poutcome.value_counts()

In [ ]:
# Combine 'unknown' and 'other' as 'other' isn't really match with either 'success' or 'failure'

In [ ]:
bank_data['poutcome']= bank_data['poutcome'].replace(['other'], 'unknown')

In [ ]:
bank_data.poutcome.value_counts()

-----------contact---------------------------

In [ ]:
# Drop 'contact', as every participant has been contacted. 

In [ ]:
bank_data.drop('contact',axis=1, inplace=True)

----------------default------------------

In [ ]:
# values for "default" : yes/no

In [ ]:
bank_data.default

bank_data['default_new'] = bank_data['default'].map({'yes':1 , 'no':0})
bank_data.drop('default',axis=1,inplace = True)
print(bank_data.default_new)

------housing-----------

In [ ]:
# values for "housing" : yes/no
bank_data.housing

In [ ]:
bank_data.housing

bank_data['housing_new'] = bank_data['housing'].map({'yes':1 , 'no':0})
bank_data.drop('housing',axis=1,inplace = True)
print(bank_data.housing_new)

---------------loan-------------

In [ ]:
# values for "loan" : yes/no
bank_data.loan

In [ ]:
bank_data['loan_new'] = bank_data['loan'].map({'yes':1 , 'no':0})
bank_data.drop('loan',axis=1,inplace = True)
print(bank_data.loan_new)

-----------------month,day-------------

In [ ]:
# day  : last contact day of the month
# month: last contact month of year
# Drop 'month' and 'day' as they don't have any intrinsic meaning

In [ ]:
bank_data.drop('month', axis=1, inplace=True)
bank_data.drop('day', axis=1, inplace=True)

-------------deposit---------------

In [ ]:
# values for "deposit" : yes/no
bank_data.deposit

In [ ]:
bank_data['deposit_new']= bank_data['deposit'].map({'yes':1, 'no':0})
bank_data.drop('deposit',axis = 1, inplace=True)
bank_data.deposit_new

-------------------pdays-------------

In [ ]:
# pdays: number of days that passed by after the client was last contacted from a previous campaign
#       -1 means client was not previously contacted

In [ ]:
bank_data.pdays

In [ ]:
print('Customers that have not been contacted before: ',len(bank_data[bank_data.pdays==-1]))
print('Maximum value on pdays: ', bank_data.pdays.max())

In [ ]:
# Map padys=-1 into a large value (10000 is used) to indicate that it is so far in the past that it has no effect

In [ ]:
bank_data.loc[bank_data['pdays']==-1, 'pdays']=10000

In [ ]:
# Create a new column: recent_pdays 
bank_data['recent_pdays'] = np.where(bank_data['pdays'], 1/bank_data.pdays, 1/bank_data.pdays)


#Drop pdays
bank_data.drop('pdays', axis=1,inplace=True)

In [ ]:
bank_data.tail()

--------------convert to dummy values------------------

In [ ]:
# Convert categorical variables to dummies

In [ ]:
bank_dummies = pd.get_dummies(bank_data, columns = ['job', 'marital', 'education', 'poutcome'])
bank_dummies.head()

In [ ]:
bank_dummies = pd.get_dummies(bank_data, columns = ['job', 'marital', 'education', 'poutcome'],prefix = ['job', 'marital', 'education', 'poutcome'])
bank_dummies.head()

In [ ]:
bank_dummies.shape

In [ ]:
bank_dummies.describe()

# Observation on whole populations

In [ ]:
## Scatterplot showing age and balance


In [ ]:
bank_dummies.plot(kind = 'scatter', x='age',y='balance')


# Across all ages, majority of people have savings of less than 20000.

In [ ]:
bank_dummies.plot(kind='hist',x = 'poutcome_success', y = 'duration')

# Analysis on people who sign up for a term deposit

In [ ]:
# People who sign up to a term deposite

In [ ]:
bank_dummies[bank_data.deposit_new == 1].describe()

In [ ]:
# People signed up to a term deposite having a personal loan (loan_new) and housing loan (housing_new)

In [ ]:
len(bank_dummies[(bank_dummies.deposit_new == 1) & (bank_dummies.loan_new) & (bank_dummies.housing_new)])


In [ ]:
# People signed up to a term deposite with a credit default 

In [ ]:
len(bank_dummies[(bank_dummies.deposit_new == 1) & (bank_dummies.default_new == 1)])



In [ ]:
# Bar chart of job Vs deposite

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(x='job', y = 'deposit_new', data = bank_data)

In [ ]:
# Bar chart of "previous outcome" Vs "call duration"

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(x='poutcome', y='duration', data=bank_data)


# Classification

In [ ]:
#make a copy

In [ ]:
bankc1 = bank_dummies

In [ ]:
#the correlation matrix

In [ ]:
cr = bankc1.corr()
cr

In [ ]:
#Heatmap

In [ ]:
plt.figure(figsize = (10,10))
cmap = sns.diverging_palette(220,10, as_cmap = True)
sns.heatmap(cr, xticklabels=cr.columns.values, yticklabels=cr.columns.values, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .82})

In [ ]:
# Extract the deposte_new column (the dependent variable)

In [ ]:
corr_deposite = pd.DataFrame(cr['deposit_new'].drop('deposit_new'))
corr_deposite.sort_values(by = 'deposit_new', ascending = False)

# Build the Model

In [ ]:
# Train-Test split: 20% test data

In [ ]:
data_drop_deposit = bankc1.drop('deposit_new', 1)
label  = bankc1.deposit_new
data_train, data_test, label_train, label_test = train_test_split(data_drop_deposit,label,test_size = 0.2, random_state = 50)

In [ ]:
# Decision tree with depth = 2

In [ ]:
dt2 = tree.DecisionTreeClassifier(random_state = 1, max_depth = 2)
dt2.fit(data_train, label_train)
dt2_score_train = dt2.score(data_train, label_train)
print('Training Score: ', dt2_score_train)
dt2_score_test = dt2.score(data_test, label_test)
print('Testing Score: ', dt2_score_test)

In [ ]:
# Decision tree with depth = 3

In [ ]:
dt3 = tree.DecisionTreeClassifier(random_state = 1, max_depth = 3)
dt3.fit(data_train, label_train)
dt3_score_train = dt3.score(data_train, label_train)
print('Training Score: ', dt3_score_train)
dt3_score_test = dt3.score(data_test, label_test)
print('Testing Score: ', dt3_score_test)

In [ ]:
# Decision tree with depth = 4


In [ ]:
dt4 = tree.DecisionTreeClassifier(random_state=1, max_depth=4)
dt4.fit(data_train, label_train)
dt4_score_train = dt4.score(data_train, label_train)
print("Training score: ",dt4_score_train)
dt4_score_test = dt4.score(data_test, label_test)
print("Testing score: ",dt4_score_test)

In [ ]:
# Decision tree with depth = 6

In [ ]:
dt6 = tree.DecisionTreeClassifier(random_state=1, max_depth=6)
dt6.fit(data_train, label_train)
dt6_score_train = dt6.score(data_train, label_train)
print("Training score: ",dt6_score_train)
dt6_score_test = dt6.score(data_test, label_test)
print("Testing score: ",dt6_score_test)

In [ ]:
# Decision tree: To the full depth

In [ ]:

dt1 = tree.DecisionTreeClassifier()
dt1.fit(data_train, label_train)
dt1_score_train = dt1.score(data_train, label_train)
print("Training score: ", dt1_score_train)
dt1_score_test = dt1.score(data_test, label_test)
print("Testing score: ", dt1_score_test)

# Compare Training and Testing scores for various tree depths used

In [ ]:
print('{:10} {:20} {:20}'.format('depth','Training Score','Testing Score'))
print('{:10} {:20} {:20}'.format('------','-------------','--------------'))
print('{:1} {:>25} {:>20}'.format(2, dt2_score_train, dt2_score_test))
print('{:1} {:>25} {:>20}'.format(3, dt3_score_train, dt3_score_test))
print('{:1} {:>25} {:>20}'.format(4, dt4_score_train, dt4_score_test))
print('{:1} {:>25} {:>20}'.format(6, dt6_score_train, dt6_score_test))
print('{:1} {:>23} {:>20}'.format("max", dt1_score_train, dt1_score_test))


In [ ]:
# Let's generate the decision tree for depth = 2
# Create a feature vector

In [ ]:
features = bankc1.columns.tolist()

# Uncomment below to generate the digraph Tree.
#tree.export_graphviz(dt2, out_file='tree_depth_2.dot', feature_names=features)

In [ ]:
# Two classes: 0 = not signed up,  1 = signed up
dt2.classes_

In [ ]:
# Create a feature vector
features = data_drop_deposit.columns.tolist()

features

In [ ]:
# Investigate most important features with depth =2

In [ ]:
dt2 = tree.DecisionTreeClassifier(random_state=1, max_depth=2)

# Fit the decision tree classifier
dt2.fit(data_train, label_train)

fi = dt2.feature_importances_


l = len(features)
for i in range(0,len(features)):
    print('{:.<20} {:3}'.format(features[i],fi[i]))

# Predictions

In [ ]:
# According to feature importance results, most importtant feature is the "Duration"
# Let's calculte statistics on Duration

In [ ]:
print("Mean duration   : ", data_drop_deposit.duration.mean())
print("Maximun duration: ", data_drop_deposit.duration.max())
print("Minimum duration: ", data_drop_deposit.duration.min())

In [ ]:
# Get a row with poutcome_success = 1
#bank_with_dummies[(bank_with_dummies.poutcome_success == 1)]
data_drop_deposit.iloc[985]

In [ ]:
# Make predictions on the test set
preds = dt2.predict(data_test)

# Calculate accuracy
print("\nAccuracy score: \n{}".format(metrics.accuracy_score(label_test, preds)))

# Make predictions on the test set using predict_proba
probs = dt2.predict_proba(data_test)[:,1]

# Calculate the AUC metric
print("\nArea Under Curve: \n{}".format(metrics.roc_auc_score(label_test, probs)))